# First Steps

In this notebook, we will guide you through the basic concepts and features of HybridAGI, a neuro-symbolic agent system that allows you to program its behavior using a graph-based prompt programming approach.

HybridAGI is designed for data scientists, prompt engineers, researchers, and AI enthusiasts who love to experiment with AI. It is a "Build Yourself" product that focuses on human creativity rather than AI autonomy. With HybridAGI, you can create AI agents that behave as expected and efficiently use any tool while controlling their long-term behavior.

In this notebook, we will cover the following topics:

- Introduction to HybridAGI and its key features
- Installation and setup of HybridAGI using virtualenv and Docker
- Overview of the Domain Specific Language (DSL) of HybridAGI and its basic syntax
- Examples of graph-based prompt programming using the DSL

By the end of this notebook, you will have an understanding of the basic concepts of HybridAGI and be able to start experimenting with it on your own.

## Setting up the Knowledge Base locally

HybridAGI works with a low-latency hybrid vector/graph database called [FalkorDB](https://www.falkordb.com/). This knowledge base should be started when you work with HybridAGI.

Start the database using docker with the following command in you terminal:

```bash
docker run -p 6379:6379 -it --rm falkordb/falkordb:edge
```

## Setting up Ollama to test your pipelines locally

To test your pipelines locally, we need to setup Ollama to run the LLMs in a separate server. 

### Linux:
Open a terminal and use the following commands:

```bash
# Install it on linux
curl -fsSL https://ollama.com/install.sh | sh
# Then launch the Ollama server with
ollama pull mistral
ollama serve
```

### Windows:
Downloads the Ollama executable [here](https://ollama.com/download/OllamaSetup.exe)

In [1]:
# Install HybridAGI package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install git+https://github.com/SynaLinks/HybridAGI.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/SynaLinks/HybridAGI.git to /tmp/pip-req-build-tlgpidwy
  Running command git clone --filter=blob:none --quiet https://github.com/SynaLinks/HybridAGI.git /tmp/pip-req-build-tlgpidwy
  Resolved https://github.com/SynaLinks/HybridAGI.git to commit 44f307d7d0226ebba9feced97db8ed6ff78884c7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# First we import the required modules
import dspy
from hybridagi import GraphProgramInterpreter
from hybridagi import SentenceTransformerEmbeddings
from hybridagi import ProgramMemory
from hybridagi import AgentState
from hybridagi.tools import SpeakTool, PredictTool

### Graph-based prompt programs introduction

HybridAGI is an agent system that introduces a `GraphProgramInterpreter`. Similar to how the Python interpreter executes a program line by line, HybridAGI's interpreter executes a graph program node by node. To store these graph programs, we use a hybrid vector/graph database called `ProgramMemory`. This memory is the sole required memory of the system and is responsible for storing Cypher Prompt Programs, such as the following example:

main.cypher:
```cypher
// @desc: The main program
CREATE
// Nodes declaration
(start:Control {name:"Start"}),
(end:Control {name:"End"}),
(answer:Action {
    name:"Answer the objective's question",
    tool:"Speak",
    prompt:"Answer the objective's question"
}),
// Structure declaration
(start)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)
```

Additionally, a dependency graph is automatically generated when loading the programs. This dependency graph enables the system to protect the main prompting mechanism and its dependencies, as well as allowing similarity-based search over the Agent graph programs.

In the provided example, we have a simple graph program consisting of three nodes: `start`, `end`, and `answer`. The `start` and `end` nodes are control nodes that denote the beginning and end of the program, respectively. The `answer` node is an action node that uses the `Speak` tool to answer the objective's question.

The edges between the nodes are defined using the `-[:NEXT]->` syntax, which indicates that the interpreter should move to the next node in the program. In this case, the interpreter will start at the `start` node, move to the `answer` node, and then finally move to the `end` node.

In [3]:

# The embeddings to use with the hybrid vector/graph store
embeddings = SentenceTransformerEmbeddings(
    dim = 384,
    model_name_or_path = "sentence-transformers/all-MiniLM-L6-v2",
)
# The LLM to use for our pipeline
llm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.settings.configure(lm=llm)

# The memory that store the Cypher programs
program_memory = ProgramMemory(
    index_name = "first_steps",
    embeddings = embeddings,
)

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
]

# The agent system
interpreter = GraphProgramInterpreter(
    program_memory = program_memory,
    tools = tools,
)

# Load the above Cypher program into memory
program_memory.add_texts(
    texts = [
"""
// @desc: The main program
CREATE
// Nodes declaration
(start:Control {name:"Start"}),
(end:Control {name:"End"}),
(answer:Action {
    name:"Answer the objective's question",
    tool:"Speak",
    prompt:"Answer the objective's question"
}),
// Structure declaration
(start)-[:NEXT]->(answer),
(answer)-[:NEXT]->(end)
"""
    ],
    ids = ["main"]
)

pred = interpreter(objective="Which city in France is known for its aerospace and robotic industry?")

/home/yoan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 --- Step 0 ---
Call Program: main
Program Purpose: Which city in France is known for its aerospace and robotic industry?
 --- Step 1 ---
Action Purpose: Answer the objective's question
Action: {
  "message": "Based on my current knowledge, Toulouse is the city in France that is well-known for its aerospace and robotic industry. Therefore, the message to send to the user would be: \"The city in France known for its aerospace and robotic industry is Toulouse.\""
}
 --- Step 2 ---
End Program: main
Final Answer:

The city in France known for its aerospace and robotic industry is Toulouse.
Chat History:

[
  {
    "role": "User",
    "message": "Which city in France is known for its aerospace and robotic industry?"
  },
  {
    "role": "AI",
    "message": "The city in France known for its aerospace and robotic industry is Toulouse."
  }
]


HybridAGI output the final result of the Agent as well as the intermediary steps and chat history. Allowing to explain and optimize its execution by tailoring custom metrics for your usecase.

In [4]:
print(pred.final_answer)

The city in France known for its aerospace and robotic industry is Toulouse.


In [5]:
print(pred.program_trace)

 --- Step 0 ---
Call Program: main
Program Purpose: Which city in France is known for its aerospace and robotic industry?
 --- Step 1 ---
Action Purpose: Answer the objective's question
Action: {
  "message": "Based on my current knowledge, Toulouse is the city in France that is well-known for its aerospace and robotic industry. Therefore, the message to send to the user would be: \"The city in France known for its aerospace and robotic industry is Toulouse.\""
}
 --- Step 2 ---
End Program: main


In [6]:
print(pred.chat_history)

[
  {
    "role": "User",
    "message": "Which city in France is known for its aerospace and robotic industry?"
  },
  {
    "role": "AI",
    "message": "The city in France known for its aerospace and robotic industry is Toulouse."
  }
]
